In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import math
from tqdm import tqdm_notebook as tqdm
from PIL import Image
%matplotlib inline

# Load & Preprocessing

In [ ]:
df=pd.read_csv('sales_merge.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.head()

## Preprocessing

In [ ]:
rct_no=list(df['rct'])
del df['rct']
# del df['chnl_dv']
df['cor']=df['cor_c'].apply(lambda x:str(x)[0])
del df['cor_c']
del df['br']
del df['loc']

In [ ]:
df.head()

In [ ]:
df['year'].unique()

In [ ]:
df=df[df['year']==2021].reset_index(drop=True)
df.head()
df.to_csv('sales_merge_fin.csv',index=False)

## NA values

In [ ]:
data=df.copy()
data.isna().sum()

In [ ]:
df_=data[~data['loc'].isna()].reset_index(drop=True)
df_.head()

In [ ]:
df_.isna().sum()

In [ ]:
df_.shape

# train-test-split
- 시계열 데이터이기 때문에 각 고객 별 마지막으로 관측된 달을 test data로 할당함

In [ ]:
df1=df_.sort_values(['cust','month']).reset_index(drop=True)
df1.head()

In [ ]:
df1.shape

In [ ]:
cust_mon=df1.groupby(['cust','month'])['hour'].count()
cust_mon

In [ ]:
cust_lst=sorted(df1['cust'].unique())
last_month=[cust_mon[i].index.max() for i in cust_lst]
last_month[:5]

In [ ]:
print(len(last_month))
print(len(cust_lst))

In [ ]:
cust_mon

In [ ]:
obs_cust=[] # 고객이 한번 관측된 경우는 test에 추가하지 않는다.
# 아마 train에서도 제거할 듯
for i in cust_lst:
  if len(cust_mon[i])==1:    
    obs_cust.append(i)
obs_cust[:10]

In [ ]:
len(obs_cust)

In [ ]:
cust_lst_=[]
last_month_=[]
for c,m in zip(cust_lst,last_month):
  if c in obs_cust:
    pass
  else:
    cust_lst_.append(c)
    last_month_.append(m)

In [ ]:
print(len(cust_lst_))
print(len(last_month_))

In [ ]:
29309-2439

In [ ]:
df1[df1['cust']=='M000034966'] # 43~51

In [ ]:
tmp=df1[['cust','month']]
mdic={}
for c,m in zip(cust_lst_,last_month_):
  mdic[c]=m
print(len(mdic))


test_indx=[]
for c in tqdm(mdic,desc='tqdm'):
  indx=tmp[(tmp['cust']==c) & (tmp['month']==mdic[c])].index
  test_indx.extend(indx)

In [ ]:
test_indx

In [ ]:
test_indx[:10]

In [ ]:
print(len(test_indx))

In [ ]:
train_indx=set(df1.index)-set(test_indx)
len(train_indx)

In [ ]:
df1.shape[0]

In [ ]:
len(train_indx)+len(test_indx)

In [ ]:
train_df=df1.iloc[list(train_indx)]
train_df

In [ ]:
train_df.to_csv('train.csv',index=False)

In [ ]:
test_df=df1.iloc[list(test_indx)]
test_df

In [ ]:
test_df.to_csv('test.csv',index=False)